<a href="https://colab.research.google.com/github/robinjameslee/CBOE-VIX-Calculation/blob/main/CBOE_VIX_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[CBOE's white paper on VIX Calculation Methodology
](https://cdn.cboe.com/api/global/us_indices/governance/Volatility_Index_Methodology_Cboe_Volatility_Index.pdf)

[CBOE Volatility Index Mathematics Methodology](https://cdn.cboe.com/api/global/us_indices/governance/Cboe_Volatility_Index_Mathematics_Methodology.pdf
)

In 1993, CBOE introduced the VIX Index. Originally designed
to measure the market’s expectation of 30-day volatility implied by at-the-money S&P 100 Index option
prices, the VIX Index soon became the premier benchmark for U.S. stock market volatility. 

It is regularly featured in the
Wall Street Journal, Barron’s, and other leading financial publications, as well as business news shows on CNBC,
Bloomberg TV, and CNN/Money, where the VIX Index is often referred to as the “fear gauge.”

The generalized formula used in the VIX Index calculation is:

VIX Index = σ * 100

T = time to expiration (in years)

F = Option-implied forward price

K0 = First Strike equal to or otherwise immediately below the foward index level, F

Ki = Strike of the ith out-of-the-money option; a call if Ki > K0, and a put if Ki < K0; both put and call if Ki = K0

Ki = interval between strike prices - half the difference between the strike on either side of Ki

Ki = (Ki+1 - Ki-1) / 2

R = Risk-free interest Rate to expiration

Q(Ki) = The midpoint of the bid-ask spread for each option with Strike Ki

In [118]:
# from google.colab import drive
# drive.mount('/content/drive')

In [119]:
import pandas as pd
import numpy as np
from datetime import datetime

In [120]:
# Read our data from the csv file
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CBOE VIX Calculation/Option Chain.csv')
df['Call_Mid'] = (df['Call Bid'] +  df['Call Ask']) / 2
df['Put_Mid'] = (df['Put Bid'] +  df['Put Ask']) / 2
df['Call_Put_Abs_Diff'] = abs(df['Call_Mid'] - df['Put_Mid'])
df['Midpoint'] = 0 # Create a midpoint column for usage later
df.head(5)

,Expiration,Days,Strike,Call Bid,Call Ask,Put Bid,Put Ask,Call_Mid,Put_Mid,Call_Put_Abs_Diff,Midpoint
0,21-Oct-22,24,800,1160.9,1164.4,0.0,0.10,1162.65,0.050,1162.600,0
1,21-Oct-22,24,900,1060.9,1064.5,0.0,0.10,1062.70,0.050,1062.650,0
2,21-Oct-22,24,1000,961.0,964.5,0.0,0.10,962.75,0.050,962.700,0
3,21-Oct-22,24,1050,911.0,914.5,0.0,0.10,912.75,0.050,912.700,0
4,21-Oct-22,24,1100,861.0,864.6,0.0,0.05,862.80,0.025,862.775,0


In [121]:
# Get the two expiration dates
near_term, next_term = df['Expiration'].unique().tolist()

# Split the dataframe by expiration
near_term_df = df[df['Expiration'] == near_term]
next_term_df = df[df['Expiration'] == next_term]

# Calculate the Single Term Variances
# Our Example:
# Current Time = 27 Sept 2022 10:45:15 am (seconds will be rounded up to the minute for calculations)
# Near Term Option = a standard SPX option expiring on 21 Oct 2022, which expires 9:30 am
# Next Term Option = a weekly SPX option expiring on 28 Oct 2022, which expires 4:00 pm
# Near Term Option Time To Maturity = 23 Days + 9 Hrs + 30 Mins + (24 Hrs - 10 Hrs - 46 mins)
# Next Term Option Time To Maturity = 30 Days + 16 Hrs + (24 Hrs - 10 Hrs - 46 mins)
# near_term_expiry_mins = (23 * 24 * 60 + 9 * 60 + 30 + (24 * 60 - 10 * 60 - 46))
# next_term_expiry_mins = (30 * 24 * 60 + 16 * 60 + (24 * 60 - 10 * 60 - 46))

# Calculate the Time to Expiration in minutes
current_time = datetime.strptime('2022-09-27 10:46:00', '%Y-%m-%d %H:%M:%S') #rounded up 10:45:15 to 10:46:00
near_term_expiry = datetime.strptime('2022-10-21 09:30:00', '%Y-%m-%d %H:%M:%S')
next_term_expiry = datetime.strptime('2022-10-28 16:00:00', '%Y-%m-%d %H:%M:%S')

near_term_expiry_mins = (near_term_expiry - current_time).total_seconds() / 60 # 34484 mins
next_term_expiry_mins = (next_term_expiry - current_time).total_seconds() / 60 # 44954 mins

one_year_in_mins = 365 * 24 * 60 # 525600 mins
thirty_days_in_mins = 30 * 24 * 60 # 43200 mins

# Calculate the Time to Expiration in Years
T1 = near_term_expiry_mins / one_year_in_mins
T2 = next_term_expiry_mins / one_year_in_mins

# Calculate the Interest Rates
# R1, R2 = Interest Rates for Near Term Option, Next Term Option respectively
R1 = 0.031664 / 100
R2 = 0.028797 / 100

In [122]:
def get_delta_K(df):
    # delta K is half of the difference between the strike prices on either side of Ki
    # if K is the upper or the lower edge, then delta K is simply the difference between Ki and the adjacent strike price
    df['delta_K'] = abs(df['Strike'].diff(periods=-1) - df['Strike'].diff(periods=1)) / 2
    df.loc[0, 'delta_K'] = abs(df.loc[0, 'Strike'] - df.loc[1, 'Strike'])
    df.loc[len(df) - 1, 'delta_K'] = abs(df.loc[len(df) - 1, 'Strike'] - df.loc[len(df) - 2, 'Strike'])
    return df

def get_contri_by_K_sum(df, K0, R, T):
    # For VIX calculation we use At-the-money-options, Out-of-the-money Calls, and Out-of-the-money Puts
    ATM = df[df['Strike'] == K0]
    ATM['Midpoint'] = (ATM['Call_Mid'] + ATM['Put_Mid']) / 2

    OTM_Call = df[df['Strike'] > K0].sort_values('Strike', ascending=True)
    OTM_Put = df[df['Strike'] < K0].sort_values('Strike', ascending=False)
    # Filter out options with 0 bid price
    # Once two options with consecutive strike prices have 0 bid price, look no further
    for index, rows in OTM_Call.iterrows():
        if rows['Call Bid'] != 0: 
            OTM_Call.loc[index, 'Midpoint'] = rows['Call_Mid']
        elif OTM_Call.loc[index + 1, 'Call Bid'] == 0:
            break
        
    for index, rows in OTM_Put.iterrows():
        if rows['Put Bid'] != 0:
            OTM_Put.loc[index, 'Midpoint'] = rows['Put_Mid']
        elif OTM_Put.loc[index + 1, 'Put Bid'] == 0:
              break

    df = pd.concat([ATM, OTM_Call, OTM_Put])
    df = df[df['Midpoint'] > 0].sort_values('Strike').reset_index()
    df = get_delta_K(df)
    df['constri_by_K'] = df['delta_K'] /  df['Strike'] ** 2 * np.exp(R * T) * df['Midpoint']
    return df['constri_by_K'].sum() * 2 / T

def square_mean_return(T, F, K0):
  return ((F / K0) - 1) ** 2 / T

In [123]:
def get_VIX():
  # Determine the option-implied forward price level, F
  # By identifying the options strike price at which the absolute difference between the call price and the put price is smallest.
  ATM_K_1 = near_term_df[near_term_df['Call_Put_Abs_Diff'] == near_term_df['Call_Put_Abs_Diff'].min()]['Strike'].min()
  ATM_K_2 = next_term_df[next_term_df['Call_Put_Abs_Diff'] == next_term_df['Call_Put_Abs_Diff'].min()]['Strike'].min()

  callPrice_1, putPrice_1 = near_term_df[near_term_df['Strike'] == ATM_K_1][['Call_Mid', 'Put_Mid']].values[0]
  callPrice_2, putPrice_2 = next_term_df[next_term_df['Strike'] == ATM_K_2][['Call_Mid', 'Put_Mid']].values[0]

  # F = Strike Price + e ** RT * (callPrice - putPrice)
  F1 = ATM_K_1 + np.exp(R1 * T1) * (callPrice_1 - putPrice_1)
  F2 = ATM_K_2 + np.exp(R2 * T2) * (callPrice_2 - putPrice_2)

  # K0 is the strike price equal to or immediately below the forward index level F
  K0_1 = near_term_df[near_term_df['Strike'] <= F1]['Strike'].max()
  K0_2 = next_term_df[next_term_df['Strike'] <= F2]['Strike'].max()

  # Calculate the sum of contribution by Strike 
  contri_by_K_sum_1 = get_contri_by_K_sum(near_term_df, K0_1, R1, T1)
  contri_by_K_sum_2 = get_contri_by_K_sum(next_term_df, K0_2, R2, T2)

  # Calculate the square of mean return divided by time to expiration
  square_mean_return_1 = square_mean_return(T1, F1, K0_1)
  square_mean_return_2 = square_mean_return(T2, F2, K0_2)

  # Calculate the variance
  var_1 = contri_by_K_sum_1 - square_mean_return_1
  var_2 = contri_by_K_sum_2 - square_mean_return_2

  # Calculate the 30-day weighted average of var_1 & var_2
  # Then take the square root and multiply by 100
  part_1 = T1 * var_1 * (next_term_expiry_mins - thirty_days_in_mins) / (next_term_expiry_mins - near_term_expiry_mins)
  part_2 = T2 * var_2 * (thirty_days_in_mins - near_term_expiry_mins) / (next_term_expiry_mins - near_term_expiry_mins)
  VIX = (part_1 + part_2) * one_year_in_mins / thirty_days_in_mins
  VIX = 100 * (VIX) ** 0.5
  return VIX

print(get_VIX())

13.927842350137642


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
